In [ ]:
import tensorflow as tf
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")


GPU is available


In [ ]:
!nvidia-smi


Tue Sep 24 12:25:08 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8              10W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install osmnx networkx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 23.8 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.4
    Uninstalling networkx-3.4:
      Successfully uninstalled networkx-3.4
  Attempting uninstall: geopandas
    Found existing installation: geopandas 1.0.1
    Uninstalling geopandas-1.0.1:
      Successfully uninstalled geopandas-1.0.1


In [ ]:
import requests
from pprint import pprint

In [ ]:
BASE_URL = 'https://nominatim.openstreetmap.org/search?format=json'

In [ ]:
!pip install geopy


In [ ]:
from geopy.geocoders import Nominatim

def get_coordinates(place_name):
    # Initialize Nominatim API
    geolocator = Nominatim(user_agent="colab_geocoder")

    # Get location
    location = geolocator.geocode(place_name)

    # Check if the location was found
    if location:
        print(f"Place: {location.address}")
        print(f"Latitude: {location.latitude}, Longitude: {location.longitude}")
        latitude=location.latitude
        longitude=location.longitude
        return latitude,longitude
    else:
        print("Place not found.")
        return None

# Example usage


In [ ]:
start_name = input("Enter the place name: ")
lat1,long1=get_coordinates(start_name)


Enter the place name: sector 43 chandigarh
Place: Sector 43, Ward 12, Chandigarh, 160043, India
Latitude: 30.7190935, Longitude: 76.7487931


In [ ]:
end_name = input("Enter the place name: ")
lat2,long2=get_coordinates(end_name)


Enter the place name: sector 17 chandigarh
Place: Sector 17, Ward 3, Chandigarh, India
Latitude: 30.7400652, Longitude: 76.7825889


In [ ]:
import requests

def get_weather(lat, lon):
    api_key = 'ef8bf71f0149b8b49ac4ec1f8ff3a105'
    url = f"http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}"
    response = requests.get(url)
    data = response.json()

    if response.status_code == 200:
      if 'weather' in data:
            weather_condition = data['weather'][0]['description']
            return weather_condition
      else:
            return "Weather data not available"
    else:
        return "Failed to fetch weather data"



# Example usage for start and end locations
weather_start = get_weather(lat1, long1)
weather_end = get_weather(lat2, long2)

print(f"Weather at Start: {weather_start}")
print(f"Weather at End: {weather_end}")


Weather at Start: clear sky
Weather at End: clear sky


In [ ]:
import folium

In [ ]:
location = float(lat1), float(long1)
location2 = float(lat2), float(long2)

m = folium.Map(location=location, zoom_start=4, width=800, height=400)

# add markers at the locations
folium.Marker(location, popup=f"Start Location: {weather_start}",icon=folium.Icon(color="green")).add_to(m)
folium.Marker(location2, popup=f"End Location: {weather_end}",icon=folium.Icon(color="red")).add_to(m)

m

In [ ]:
import osmnx as ox
import networkx as nx

In [ ]:
start_name_waypoint= input("Enter the place name: ")
latt1,lon1=get_coordinates(start_name_waypoint)

Enter the place name: sector 22 chandigarh
Place: Sector 22, Ward 3, Chandigarh, India
Latitude: 30.7334256, Longitude: 76.7713451


In [ ]:
start_name_waypoint = input("Enter the place name: ")
latt2,lon2=get_coordinates(start_name_waypoint)

Enter the place name: sector 27 chandigarh
Place: Sector 27, Ward 18, Chandigarh, 160027, India
Latitude: 30.7227459, Longitude: 76.7984247


In [ ]:
wayp1=[latt1,lon1]
wayp2=[latt2,lon2]
m

In [ ]:
folium.Marker(location, popup=f"Start Location: {weather_start}",icon=folium.Icon(color="green")).add_to(m)
folium.Marker(location2, popup=f"End Location: {weather_end}",icon=folium.Icon(color="red")).add_to(m)
folium.Marker(wayp1, popup=f"way 1: {weather_start}",icon=folium.Icon(color="blue")).add_to(m)
folium.Marker(wayp2, popup=f"way 2: {weather_end}",icon=folium.Icon(color="blue")).add_to(m)
m

In [ ]:
def weather_affected_weight(edge_data, weather_condition):
    """
    Adjusts the weight of an edge based on the weather condition.
    Different weather conditions have different impact multipliers.
    """
    base_length = edge_data['length']  # Base length of the road segment in meters

    # Define weather impact multipliers
    weather_multipliers = {
        'clear': 1.0,            # No impact
        'clouds': 1.0,           # No significant impact
        'rain': 1.2,             # 20% increase for rain
        'drizzle': 1.1,          # 10% increase for drizzle
        'thunderstorm': 1.5,     # 50% increase for thunderstorms
        'snow': 1.5,             # 50% increase for snow
        'mist': 1.1,             # 10% increase for mist
        'fog': 1.1,              # 10% increase for fog
        'haze': 1.1,             # 10% increase for haze
        'smoke': 1.1             # 10% increase for smoke
    }

    multiplier = weather_multipliers.get(weather_condition, 1.0)  # Default to 1.0 if condition not listed
    adjusted_weight = base_length * multiplier

    return adjusted_weight

In [ ]:
def apply_weather_weights(G, weather_conditions, locations, radius=1000):
    """
    Applies weather-affected weights to the graph G based on multiple weather conditions
    and their corresponding locations.

    Parameters:
    - G: The road network graph.
    - weather_conditions: A list of tuples containing (weather_condition, target_lat, target_lon).
    - locations: A list of tuples containing (lat, lon) for which weather is fetched.
    - radius: The radius in meters within which to apply the weather effect.

    Returns:
    - G: The modified graph with updated 'weather_weight' attributes.
    """
    for condition, target_lat, target_lon in weather_conditions:
        for u, v, key, data in G.edges(keys=True, data=True):
            # Get the coordinates of the edge's nodes
            node_u = G.nodes[u]
            node_v = G.nodes[v]

            # Calculate distance from target location to the edge's nodes
            dist_to_u = ox.distance.euclidean_dist_vec(node_u['y'], node_u['x'], target_lat, target_lon)
            dist_to_v = ox.distance.euclidean_dist_vec(node_v['y'], node_v['x'], target_lat, target_lon)

            # Apply weather effect only if the edge is within the target radius
            if dist_to_u <= radius or dist_to_v <= radius:
                # If multiple weather conditions affect the same edge, choose the highest multiplier
                current_weight = data.get('weather_weight', data['length'])
                new_weight = weather_affected_weight(data, condition)
                # Assign the maximum of the current and new weight to handle overlapping conditions
                data['weather_weight'] = max(current_weight, new_weight)
            else:
                # If no weather condition affects this edge, ensure 'weather_weight' is set to base length
                if 'weather_weight' not in data:
                    data['weather_weight'] = data['length']
    return G

In [ ]:
# Start and End Locations
start_location = [lat1, long1]  # e.g., [40.7128, -74.0060]
end_location = [lat2, long2]    # e.g., [34.0522, -118.2437]

# Waypoints for alternate paths
waypoint1 = [latt1, lon1]  # e.g., [41.8781, -87.6298]
waypoint2 = [latt2, lon2]  # e.g., [29.7604, -95.3698]

# ------------------------------
# Step 3: Fetch Weather Conditions
# ------------------------------

# Fetch weather conditions for start and end locations
weather_start = get_weather(start_location[0], start_location[1])
weather_end = get_weather(end_location[0], end_location[1])

print(f"Weather at Start: {weather_start}")
print(f"Weather at End: {weather_end}")

# You can also fetch weather for waypoints if needed
# weather_waypoint1 = get_weather(waypoint1[0], waypoint1[1])
# weather_waypoint2 = get_weather(waypoint2[0], waypoint2[1])
# print(f"Weather at Waypoint 1: {weather_waypoint1}")
# print(f"Weather at Waypoint 2: {weather_waypoint2}")

# ------------------------------
# Step 4: Build the Road Network Graph
# ------------------------------

# Get the graph for the area around the start location
G = ox.graph_from_point(start_location, dist=50000, network_type='drive')

# ------------------------------
# Step 5: Apply Weather Weights to the Graph
# ------------------------------

# Define weather conditions and their target locations
# For this example, we'll apply weather effects to both start and end locations
weather_conditions = [
    (weather_start, start_location[0], start_location[1]),
    (weather_end, end_location[0], end_location[1])
]

# Apply weather weights to the graph
radius = 1000  # Radius in meters to apply weather effects

G = apply_weather_weights(G, weather_conditions, locations=[start_location, end_location], radius=radius)

# ------------------------------
# Step 6: Find Nearest Nodes
# ------------------------------

# Find the nearest nodes to the start, end, and waypoint points
start_node = ox.distance.nearest_nodes(G, X=start_location[1], Y=start_location[0])
end_node = ox.distance.nearest_nodes(G, X=end_location[1], Y=end_location[0])
waypoint1_node = ox.distance.nearest_nodes(G, X=waypoint1[1], Y=waypoint1[0])
waypoint2_node = ox.distance.nearest_nodes(G, X=waypoint2[1], Y=waypoint2[0])

# ------------------------------
# Step 7: Initialize the Folium Map
# ------------------------------

# Initialize the folium map centered at the start location
m = folium.Map(location=start_location, zoom_start=13)

# ------------------------------
# Step 8: Define Route Plotting Function
# ------------------------------

def plot_route(route, color, map_obj, label):
    """
    Plots a route on the folium map and prints its length.

    Parameters:
    - route: List of node IDs representing the route.
    - color: Color of the route line.
    - map_obj: The folium Map object.
    - label: Label for the route (used in the print statement).

    Returns:
    - route_length: Length of the route in kilometers.
    """
    ox.plot_route_folium(G, route, route_map=map_obj, color=color, opacity=0.7, weight=5, tooltip=label)
    route_length = sum(ox.utils_graph.get_route_edge_attributes(G, route, 'length'))
    print(f"{label} distance by road: {route_length/1000:.2f} km")
    return route_length

# ------------------------------
# Step 9: Calculate and Plot Alternate Routes
# ------------------------------

# Route via Waypoint 1
route_via_waypoint1_part1 = nx.shortest_path(G, source=start_node, target=waypoint1_node, weight='weather_weight', method='dijkstra')
route_via_waypoint1_part2 = nx.shortest_path(G, source=waypoint1_node, target=end_node, weight='weather_weight', method='dijkstra')
# Merge the two parts, avoiding duplicate waypoint node
route_via_waypoint1 = route_via_waypoint1_part1 + route_via_waypoint1_part2[1:]
plot_route(route_via_waypoint1, color="purple", map_obj=m, label="Route via Waypoint 1")

# Route via Waypoint 2
route_via_waypoint2_part1 = nx.shortest_path(G, source=start_node, target=waypoint2_node, weight='weather_weight', method='dijkstra')
route_via_waypoint2_part2 = nx.shortest_path(G, source=waypoint2_node, target=end_node, weight='weather_weight', method='dijkstra')
# Merge the two parts, avoiding duplicate waypoint node
route_via_waypoint2 = route_via_waypoint2_part1 + route_via_waypoint2_part2[1:]
plot_route(route_via_waypoint2, color="orange", map_obj=m, label="Route via Waypoint 2")

# ------------------------------
# Step 10: Save and Download the Map
# ------------------------------
# Step 7: Initialize the Folium Map with Markers
# Initialize the folium map centered at the start location


# Add markers for the start location, end location, and waypoints
folium.Marker(start_location, popup=f"Start Location: {weather_start}",icon=folium.Icon(color="green")).add_to(m)
folium.Marker(end_location, popup=f"End Location: {weather_end}",icon=folium.Icon(color="red")).add_to(m)
folium.Marker(waypoint1, popup=f"way 1",icon=folium.Icon(color="blue")).add_to(m)
folium.Marker(waypoint2, popup=f"way 2",icon=folium.Icon(color="blue")).add_to(m)

# Save the map to an HTML file
m.save('map.html')
from google.colab import files
# Download the HTML file to your local machine
files.download('map.html')

Weather at Start: clear sky
Weather at End: clear sky


/usr/local/lib/python3.10/dist-packages/osmnx/graph.py:191: FutureWarning: The expected order of coordinates in `bbox` will change in the v2.0.0 release to `(left, bottom, right, top)`.
  G = graph_from_bbox(
<ipython-input-17-d04fea02405b>:22: FutureWarning: The `euclidean_dist_vec` function has been renamed `euclidean`. Calling `euclidean_dist_vec` will raise an error starting in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  dist_to_u = ox.distance.euclidean_dist_vec(node_u['y'], node_u['x'], target_lat, target_lon)
<ipython-input-17-d04fea02405b>:23: FutureWarning: The `euclidean_dist_vec` function has been renamed `euclidean`. Calling `euclidean_dist_vec` will raise an error starting in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  dist_to_v = ox.distance.euclidean_dist_vec(node_v['y'], node_v['x'], target_lat, target_lon)
<ipython-input-26-a2459d2593d0>:83: FutureWarning: 

Route via Waypoint 1 distance by road: 5.81 km
Route via Waypoint 2 distance by road: 9.29 km


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>